In [1]:
import json
import sys
import os
sys.path.append('/Users/gisaac/Documents/GitHub/river_flow_notebooks/')

with open("scan_09408150.json") as f:
    data = json.load(f)

In [2]:
from scraping_functions import HydroScraper

In [6]:
from datetime import datetime
h = HydroScraper(datetime(1988, 1, 1), datetime(2024, 1, 30), "scan_09408150.json", "weather_asos_final")

Getting request from USGS
http://waterservices.usgs.gov/nwis/iv/?format=rdb,1.0&sites=09408150&startDT=1988-01-01&endDT=2024-01-30&parameterCd=00060,00065,00045&siteStatus=all
Request finished


/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:112: DtypeWarning: Columns (1,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep="\t")
/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["datetime"] = df["datetime"].map(lambda x: old_timezone.localize(datetime.strptime(x, "%Y-%m-%d %H:%M")).astimezone(new_timezone))
/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

Getting request from ASOS
https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=SGU&data=tmpf&data=dwpf&data=relh&data=feel&data=sknt&data=sped&data=alti&data=mslp&data=drct&data=ice_accretion_1hr&data=p01m&data=vsby&data=gust&data=skyc1&data=peak_wind_gust&data=snowdepth&year1=1988&month1=1&day1=1&year2=2024&month2=2&day2=1&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=M&trace=T&direct=no&report_type=3&report_type=4


/Users/gisaac/Documents/GitHub/river_flow_notebooks/weather_scraping_functions.py:33: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path) # , parse_dates=['valid']


       station             valid   tmpf   dwpf   relh   feel  sknt  sped  \
0          SGU  1988-01-03 16:00  30.20      M      M      M  0.00  0.00   
1          SGU  1988-01-03 17:00  35.60      M      M      M  0.00  0.00   
2          SGU  1988-01-03 18:00  35.60      M      M      M  0.00  0.00   
3          SGU  1988-01-03 19:00  35.60      M      M      M  0.00  0.00   
4          SGU  1988-01-04 00:00  41.00      M      M      M  0.00  0.00   
...        ...               ...    ...    ...    ...    ...   ...   ...   
412982     SGU  2024-01-30 03:56  52.00  41.00  66.00  52.00  0.00  0.00   
412983     SGU  2024-01-30 04:56  50.00  42.00  73.87  50.00  0.00  0.00   
412984     SGU  2024-01-30 05:56  50.00  40.00  68.37  50.00  3.00  3.45   
412985     SGU  2024-01-30 06:56  49.00  39.00  68.25  48.12  3.00  3.45   
412986     SGU  2024-01-30 07:56  48.00  37.00  65.51  46.98  3.00  3.45   

         alti     mslp    drct ice_accretion_1hr  p01m   vsby gust skyc1  \
0       30.

/Users/gisaac/anaconda3/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


LoadJob<project=hydro-earthnet-db, location=EU, id=e5e9cbd2-f966-4723-bce3-fd2cbff09d82>
ASOS data written to BigQuery


In [ ]:
h.combine_data()

In [ ]:
h.joined_df

In [ ]:
h.combine_snotel_with_df()

In [ ]:
h.final_df.to_csv("s11059300_89_24.csv")

In [ ]:
import pandas as pd

In [ ]:
!pwd

In [ ]:
sentinel_df = pd.read_csv("/Users/gisaac/Downloads/sent_data.csv")

In [ ]:
h.combine_sentinel(sentinel_df)
h.final_df

In [ ]:
def combine_sentinel(final_df, sentinel_df, tile):
    """ to combine the Sentinel data with the joined ASOS, USGS, and SNOTEL data.
    """
    sentinel_df = sentinel_df[sentinel_df["mgrs_tile"]==tile] 
    sentinel_df = sentinel_df[["sensing_time", "base_url"]]
    sentinel_df["SENSING_TIME"] = pd.to_datetime(sentinel_df["sensing_time"], utc=True, format='mixed').round('60min')
    final_df =final_df.merge(sentinel_df, left_on="hour_updated", right_on="SENSING_TIME", how="left")
    return final_df

fin_df = combine_sentinel(h.final_df, sentinel_df, "11SMT")
fin_df

In [ ]:
fin_df.dropna(subset=["base_url"])

In [ ]:
 sentinel_df_11 = sentinel_df[sentinel_df["mgrs_tile"]=="11SMT"] 

In [ ]:
sentinel_df_11

In [ ]:
find = fin_df

In [ ]:
find.to_csv("final_11059300.csv")